# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../output_data/cities.csv")

# First overview the data set - how many factors, etc.
cities.head()

,city,cloudiness,Country,Date,Humidity,lat,lon,Max Temp,Wind Speed
0,Honiara,20,SB,1585000092,94,-9.43,159.95,26.00,1.02
1,Ponta do Sol,40,PT,1585000093,67,32.67,-17.10,17.00,6.70
2,Pevek,100,RU,1585000093,86,69.70,170.31,-7.43,6.64
3,Wonthaggi,87,AU,1585000094,75,-38.60,145.59,14.44,0.45
4,Ushuaia,75,AR,1585000095,67,-54.80,-68.30,14.00,9.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities[["lat", "lon"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
cities_complete = cities.dropna()
humidity = cities["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
cities_ideal = cities.loc[ cities['Max Temp'] <= 25]
cities_ideal = cities_ideal.loc[ cities_ideal['Max Temp'] > 5]
cities_ideal = cities_ideal.loc[ cities_ideal['Wind Speed'] < 10]
cities_ideal = cities_ideal.loc[ cities_ideal['cloudiness'] < 50]
cities_ideal


,city,cloudiness,Country,Date,Humidity,lat,lon,Max Temp,Wind Speed
1,Ponta do Sol,40,PT,1585000093,67,32.67,-17.10,17.00,6.70
8,Los Llanos de Aridane,20,ES,1585000097,55,28.66,-17.92,19.00,9.80
18,Carnarvon,0,AU,1585000104,69,-24.87,113.63,24.00,3.10
27,Shingū,0,JP,1585000115,53,33.73,135.98,7.77,4.55
29,Chui,41,UY,1585000116,65,-33.70,-53.46,23.77,6.19
...,...,...,...,...,...,...,...,...,...
499,Arroyo de la Encomienda,6,ES,1585000535,87,41.61,-4.80,10.56,4.10
502,Katsuura,20,JP,1585000539,48,35.13,140.30,7.00,9.30
507,Zhuhai,40,CN,1585000543,94,22.28,113.57,22.78,5.10
534,Eureka,40,US,1585000339,62,40.80,-124.16,12.78,2.80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
import json
hotel_df = []
locations1 = cities_ideal[["lat", "lon"]].astype(float)
for index, row in cities_ideal.iterrows():
    try:
        target_lat = float(row["lat"])
        target_lon = float(row["lon"])
        target_coordinates = row["lat"], row["lon"]
        #target_coordinates = "32.67, -17.1"
        #print (target_coordinates)
        target_search = "Hotel"
        target_radius = 5000
        target_type = "lodging"
        params = {

            "keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
        }
        
        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362"
        base_url1 = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=" + str (row["lat"]) +"," + str(row["lon"]) 
        
        # run a request using our params dictionary
        response = requests.get(base_url1, params=params)

        # convert response to json
        places_data = response.json()

        print(places_data["results"][0]["name"])
        hotel_df.append(places_data["results"][0]["name"])
    except IndexError:
        hotel_df.append("NA")
        print("Hotel not found. Skipping...")
print(hotel_df)
print (len(hotel_df)) 


Hotel Da Vila
Hotel Hacienda de Abajo
Hospitality Carnarvon
Gran Hotel
Bertelli Chuí Hotel
HOTEL VALENCIA
Jinrui Hotel
Hotel Buganville Express
FM7 Resort Hotel Jakarta Airport
Hotel Brisas De Comarapa
Hôtel Djamtum
The Landing Hotel
Sands Hotel Margate
Hotel Xaguate
Shimoda Tokyu Hotel
Vinpearl Hotel Quảng Bình
The Woodbridge
Quest Hotel San Denpasar
Hotel Grand Chancellor Hobart
Shinra
The Harbourview
Hotel Castelo
Desert Gardens Hotel - Ayers Rock Resort
Hotel not found. Skipping...
Wajiima Onsen Yashio
Nino village
Reflections Killalea Reserve
Kasa Resort
Crest Hotel
Quality Inn
Odar kanawal
Pousada Kactus
Hotel Torre Praia
Hotel da Canoa
Lilin Lovina Beach Hotel
Hotel not found. Skipping...
Hotel Karibu
7 Days Premium
Mercure Alice Springs Resort
Hotel Memory
Golden Tulip Porto Vitória
Napakuang Resort
Yadav Hotel
Desert Rendezvous
Gardenia
Mission Of The Sun Hotel
Hotel not found. Skipping...
Hotel VIP Praia
Hotel Frontera La Quiaca
Rollins Hotel
Yeppoon Beach House
Hotel Playa E

In [6]:
cities_ideal["Hotel Name"] = hotel_df
cities_ideal


,city,cloudiness,Country,Date,Humidity,lat,lon,Max Temp,Wind Speed,Hotel Name
1,Ponta do Sol,40,PT,1585000093,67,32.67,-17.10,17.00,6.70,Hotel Da Vila
8,Los Llanos de Aridane,20,ES,1585000097,55,28.66,-17.92,19.00,9.80,Hotel Hacienda de Abajo
18,Carnarvon,0,AU,1585000104,69,-24.87,113.63,24.00,3.10,Hospitality Carnarvon
27,Shingū,0,JP,1585000115,53,33.73,135.98,7.77,4.55,Gran Hotel
29,Chui,41,UY,1585000116,65,-33.70,-53.46,23.77,6.19,Bertelli Chuí Hotel
...,...,...,...,...,...,...,...,...,...,...
499,Arroyo de la Encomienda,6,ES,1585000535,87,41.61,-4.80,10.56,4.10,AC Hotel by Marriott Palacio de Santa Ana
502,Katsuura,20,JP,1585000539,48,35.13,140.30,7.00,9.30,Katsuura Hilltop Hotel & Residence
507,Zhuhai,40,CN,1585000543,94,22.28,113.57,22.78,5.10,Pullman Zhuhai
534,Eureka,40,US,1585000339,62,40.80,-124.16,12.78,2.80,Best Western Plus Bayshore Inn


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cities_ideal.iterrows()]
locations = cities_ideal[["lat", "lon"]]

In [8]:
# Add marker layer ontop of heat map
import gmplot
marker_locations = locations
locations = cities[["lat", "lon"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
cities_complete = cities.dropna()
humidity = cities["Humidity"].astype(float)

fig = gmaps.figure()
#marker_layer = gmaps.marker_layer(plant_locations, info_box_content=plant_info)
markers = gmaps.marker_layer(marker_locations, info_box_content= hotel_info)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))